In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output

import requests
import os

!pip install sodapy -q
from sodapy import Socrata

In [ ]:
## Reference: http://holowczak.com/getting-started-with-nyc-opendata-and-the-socrata-api/5/
data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
app_token='ERDGUvZUDVHLJnGt3M7JwSJR3'   # The app token created in the prior steps
client = Socrata(data_url, app_token)      # Create the client to point to the API endpoint
client.timeout = 60 # Set the timeout to 60 seconds    

In [ ]:
data_page_url_df = pd.read_csv('/content/drive/MyDrive/metavis/data_page_url_df.csv')
data_page_url_df = data_page_url_df.drop_duplicates()

In [ ]:
def sorted_unique_values(data, col):
  return sorted(data[col].dropna().unique().tolist())

In [ ]:
metadata_list = []
error_list = []
for data_uuid in tqdm(sorted_unique_values(data_page_url_df, 'data_uuid')):
  try:
    metadata_list.append( client.get_metadata(data_uuid) )
  except Exception as e:
    error_list.append( (data_uuid, e) ) 

In [ ]:
metadata_df = pd.DataFrame(metadata_list)
metadata_df = metadata_df.rename(columns = {'id':'data_uuid'})
metadata_df = pd.merge(metadata_df, data_page_url_df[['data_uuid','data_page_url']], how='left')
metadata_df_columns = list(metadata_df.columns)
metadata_df_columns.remove('data_uuid')
metadata_df_columns = ['data_uuid']+sorted(metadata_df_columns)
metadata_df = metadata_df[metadata_df_columns]

In [ ]:
# metadata_df.to_csv(X'/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv',index=False)

In [ ]:
metadata_df = pd.read_csv('/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv')
metadata_df['attachments'] = metadata_df['metadata'].apply(eval).apply(lambda x: x.get('attachments',[]))

for _, row in tqdm(metadata_df.iterrows()):
  
  data_uuid = row['data_uuid']

  data_attachment_folder = '/content/drive/MyDrive/metavis/data_attachments/'+data_uuid
  if not os.path.exists(data_attachment_folder):
      os.mkdir(data_attachment_folder)
  
  attachment_info_list = row['attachments']

  for info in attachment_info_list:

    if 'assetId' in info.keys():
      r = requests.get('https://data.cityofnewyork.us/api/views/'+data_uuid+'/files/'+info['assetId'])  
      attachment_file_path = data_attachment_folder +'/'+ info['filename']

      if not os.path.exists(attachment_file_path):
        with open(attachment_file_path, 'wb') as f:
            f.write(r.content)